In [ ]:
# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from collections import defaultdict
import math
import random
from copy import deepcopy
from TeamMaking_module import sort_by_todofuken, sameTeam_count, womenLess_count, GroupingProblem

tdfk = ['北海道', '青森', '岩手', '宮城', '秋田', '山形', '福島',
        '茨城', '栃木', '群馬', '埼玉', '千葉', '東京', '神奈川',
        '新潟', '富山', '石川', '福井', '山梨', '長野', '岐阜',
        '静岡', '愛知', '三重', '滋賀', '京都', '大阪', '兵庫',
        '奈良', '和歌山', '鳥取', '島根', '岡山', '広島', '山口', 
        '徳島', '香川', '愛媛', '高知', '福岡', '佐賀', '長崎',
        '熊本', '大分', '宮崎', '鹿児島', '沖縄', '海外', 'なし']

In [ ]:
# 前処理
# データ読み込み

raw_data = pd.read_excel("./data/participant_data_all.xlsx")
# raw_data.drop('出身', axis=1, inplace=True)

# 余分な半角スペースや全角スペースを削除する
raw_data["氏名"] = raw_data["氏名"].str.replace('\s', '', regex=True).values

remove_data = pd.read_excel("./data/グループ分け除外リスト.xlsx")
print(raw_data.shape)

In [ ]:
A_coordinator_list = pd.read_csv("./data/assignmentA_data.csv").loc[:, '氏名'].tolist()
B_coordinator_list = pd.read_csv("./data/assignmentB_data.csv").loc[:, '氏名'].tolist()
A_coordinator_list.extend(['春日原大地', '濱野将貴'])
B_coordinator_list.extend(['中尾圭吾', '森田啓介'])

A_coordinator = raw_data.query("氏名 in @A_coordinator_list")
B_coordinator = raw_data.query("氏名 in @B_coordinator_list")

other = raw_data.query("'その他' in 参加場所")

remove_list = remove_data.loc[:, '氏名'].tolist() + A_coordinator_list + B_coordinator_list + other.loc[:, '氏名'].to_list()
data = raw_data.query("氏名 not in @remove_list")

In [ ]:
other

In [ ]:
# 重複している人を削除する
unique_data = data.drop_duplicates(subset='氏名')

In [ ]:
fixed_data = unique_data.copy()
fixed_data = fixed_data.sort_values('性別')
fixed_data.reset_index(inplace=True, drop=True)

In [ ]:
fixed_data = sort_by_todofuken(fixed_data, tdfk=tdfk)

In [ ]:
fixed_data.value_counts(subset='性別')

In [ ]:
B_coordinator

In [ ]:
# 男性をAとBに分ける(分ける人数は資料を参照)
male_data = fixed_data[fixed_data["性別"] == "男"]
male_num = len(male_data)
male_teamA = male_data.iloc[:111, :]
male_teamB = male_data.iloc[111:, :]

# 女性をAとBに分ける
female_data = fixed_data[fixed_data["性別"] == "女"]
female_num = len(female_data)
female_teamA = female_data.iloc[:54, :]
female_teamB = female_data.iloc[54:, :]

print(f'Male number is {len(male_teamA)}')
print(f'Female number is {len(female_teamA)}')

In [ ]:
other_male = other.query('性別 == "男"')
other_female = other.query('性別 == "女"')

B_coordinator_male = B_coordinator.query('性別 == "男"')
B_coordinator_female = B_coordinator.query('性別 == "女"')

male_teamB = pd.concat([male_teamB, B_coordinator_male])
female_teamB = pd.concat([female_teamB,  B_coordinator_female])

In [ ]:
B_coordinator

In [ ]:
male_teamB.shape, female_teamB.shape

In [ ]:
totalMember_num = len(female_teamB) + len(male_teamB)
print(f'Number of Team B member is {totalMember_num}')
team_num = 36
if totalMember_num % team_num == 0:
    teamMember_num = totalMember_num // team_num
    shortage = 0
else:
    teamMember_num = totalMember_num // team_num + 1
    shortage = team_num * teamMember_num - totalMember_num
totalMember_num += (shortage)
print(f'Member number is `{totalMember_num}') 
print(f'Team number is `{team_num}')

In [ ]:
# 同じチームになったことがあるか記録する
sameTeam_list = np.zeros((totalMember_num, totalMember_num))

# 女性が少ないチームになったことがあるかを記録する
womenLess_list = np.zeros(totalMember_num)

In [ ]:
# 男性、女性、ダミーの順番で並んだdataframeを作成する
dammyDF = pd.DataFrame([['ダミー', 'なし', 'なし', 'なし', 'なし'] for i in range(shortage)], columns=fixed_data.columns)
outputDF = pd.concat([male_teamB, female_teamB, dammyDF]).reset_index(drop=True)
outputDF = sort_by_todofuken(outputDF)
member_list = outputDF.values.tolist()

In [ ]:
print(f'Team number is {team_num}')
print(f'Total number of member is {totalMember_num}')

In [ ]:
# 1回目のチームを作成する（チーム座談会）
# 男、女、ダミーの順で並んでいるため、前から順にチーム番号を振っていく
teamList = np.tile(np.arange(team_num), (teamMember_num,1))
memberList = np.arange(totalMember_num).reshape(teamMember_num, team_num)
outputDF["チーム座談会_班番号"] = teamList.ravel()
# 同じチームになった回数をカウントする
sameTeam_list = sameTeam_count(sameTeam_list, memberList.T)
# 女性が少ないチームになった回数をカウントする
gender_count_by_team = [defaultdict(int) for _ in range(team_num)]
for i in range(totalMember_num):
    gender_count_by_team[teamList.ravel()[i]][outputDF["性別"].values[i]] += 1
womenLess_list = womenLess_count(womenLess_list, teamList.ravel(), gender_count_by_team)

In [ ]:
memberList

In [ ]:
# チームを作成する（ランダム座談会用）
# 最適化を使用して、同じチームが発生しないようにする
random.seed(95)
init_state = [list(teamList.ravel()), gender_count_by_team, memberList.T.tolist()]
prob = GroupingProblem(init_state, member_list, sameTeam_list, womenLess_list, totalMember_num, team_num)
prob.steps = 10**4 * 3
prob.copy_strategy = "deepcopy"
state, e = prob.anneal()
sameTeam_list = sameTeam_count(sameTeam_list, prob.state[2])
womenLess_list = womenLess_count(womenLess_list, prob.state[0], prob.state[1])
outputDF["ランダム座談会_班番号"] = prob.state[0]

In [ ]:
outputDF

In [ ]:
# 作成する（チーム対抗レク）
team_zadankai = outputDF['チーム座談会_班番号']
for i in range(18, 36):
    team_zadankai = team_zadankai.replace(i, i-18)

# print(team_zadankai)
outputDF['チーム対抗レク_班番号'] = team_zadankai
outputDF['チーム対抗レク_班番号'] = outputDF['チーム対抗レク_班番号'].astype(int)

team_num = 18
team_recreation_List = np.ndarray((team_num, totalMember_num//team_num), dtype=int)
for i in range(team_num):
    team_recreation = outputDF.query(f'チーム対抗レク_班番号 == {i}').index.to_numpy()
    # print(team_recreation)
    team_recreation_List[i] = team_recreation

# 同じチームになった回数をカウントする
sameTeam_list = sameTeam_count(sameTeam_list, team_recreation_List)


# 女性が少ないチームになった回数をカウントする
gender_count_by_team = [defaultdict(int) for _ in range(team_num)]
for i in range(team_num):
    for j in range(totalMember_num//team_num):
        gender_count_by_team[i][outputDF['性別'].values[i+j*18]] += 1
womenLess_list = womenLess_count(womenLess_list, np.array(outputDF['チーム対抗レク_班番号']).ravel(), gender_count_by_team)

In [ ]:
outputDF = pd.concat([outputDF, pd.Series(name='個人対抗レク_班番号', dtype=np.float64)], axis=1)

In [ ]:
# 作成する（個人対抗レク）
for i in range(len(outputDF)):
    outputDF.loc[i, '個人対抗レク_班番号'] = int(29-(i%29))

In [ ]:
outputDF

In [ ]:
# 重複している回数を計算する
np.count_nonzero(sameTeam_list >= 2) // 2

In [ ]:
# 女性少数チームの回数を計算する
np.count_nonzero(womenLess_list == 3) // 2

## csvファイルとして出力

In [ ]:
teamB = deepcopy(outputDF)
teamB["グループ"] = "B"

In [ ]:
# 出力するチーム名を適切な形に変化させる
apply_list =['チーム座談会_班番号', 'チーム対抗レク_班番号', 'ランダム座談会_班番号']
for l in apply_list:
    teamB[l] = teamB[l].apply(lambda n: n+1)

In [ ]:
teamB.to_excel("./out/B_班番号.xlsx")

In [ ]:
copyB = deepcopy(teamB.loc[:,['氏名', '個人対抗レク_班番号']])
copyB["何人目"] = copyB.groupby('個人対抗レク_班番号').cumcount()+1
B_data = copyB.pivot(index='個人対抗レク_班番号', columns='何人目', values='氏名')

In [ ]:
apply_list =['チーム座談会_班番号', 'ランダム座談会_班番号', 'チーム対抗レク_班番号', '個人対抗レク_班番号']
with pd.ExcelWriter("./out/B_班員.xlsx") as writer:
    for l in apply_list:
        copyB = deepcopy(teamB.loc[:,['氏名', l]])
        copyB["何人目"] = copyB.groupby(l).cumcount()+1
        B_data = copyB.pivot(index=l, columns='何人目', values='氏名')
        B_data.fillna('ダミー', inplace=True)
        B_data.to_excel(writer, sheet_name=l)